In [1]:
!git clone https://github.com/Rope-player/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 548 (delta 5), reused 162 (delta 0), pack-reused 374
Receiving objects: 100% (548/548), 50.13 MiB | 18.53 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [3]:
%cd "2_objectdetection"

/content/pytorch_advanced/2_objectdetection


In [11]:
import os.path as osp
import random
import time
import os
import urllib.request
import zipfile
import tarfile

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [12]:
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [13]:
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

In [14]:
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    tar = tarfile.TarFile(target_path)
    tar.extractall(data_dir)
    tar.close()

In [15]:
url = "https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth"
target_path = os.path.join(weights_dir, "vgg16_reducedfc.pth") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)

In [16]:
url = "https://s3.amazonaws.com/amdegroot-models/ssd300_mAP_77.43_v2.pth"
target_path = os.path.join(weights_dir, "ssd300_mAP_77.43_v2.pth") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)

In [17]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 학습 및 검증 실시

**학습 프로그램의 흐름 개요**
1. 데이터 로더 만들기
2. 네트워크 모델 만들기
3. 손실함수 정의
4. 최적화 기법 설정
5. 학습 및 검증 실시

## 데이터 로더 작성

In [20]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# 파일 경로 리스트 취득
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# Dataset 작성
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)
input_size = 300

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader 작성
batch_size = 32

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

## 네트워크 모델 작성

In [21]:
from utils.ssd_model import SSD

# SSD300 설정
ssd_cfg = {
    'num_classes': 21,                           # 총 클래스의 수
    'input_size': 300,                           # 화상 입력 크기
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],       # 출력할 DBox의 화면비 종류
    'feature_maps': [38, 19, 10, 5, 3, 1],       # 각 source 화상 크기
    'steps': [8, 16, 32, 64, 100, 300],          
    'min_sizes': [30, 60, 111, 162, 213, 264],   # DBOX 크기(최소)
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOX 크기(최대)
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSD 네트워크 모델
net = SSD(phase="train", cfg=ssd_cfg)

# SSD 초기 가중치 설정
# ssd의 vgg 가중치 로드
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssd의 기타 네트워크 가중치는 He의 초기자로 초기화


def weights_init(m):
  if isinstance(m, nn.Conv2d):
    init.kaiming_normal_(m.weight.data)
    if m.bias is not None:
      nn.init.constant_(m.bias, 0.0)


# He의 초기화 적용
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPU 사용가능 여부 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치：", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다.')

사용 중인 장치： cpu
네트워크 설정 완료: 학습된 가중치를 로드했습니다.


## 손실 함수 및 최적화 기법 설정

In [ ]:
from utils.ssd_model import MultiBoxLoss

# 손실 함수 설정
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 최적화 기법 설정
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

## 학습 및 검증

학습 및 검증을 시행사는 `train_model` 함수 실행. 10에폭에 한 번으로 검증.

학습 및 검증의 손실 값은 각 에폭마다 `log_output.csv`에 저장함.

In [22]:
# 모델을 학습시키는 함수 
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("사용 중인 장치：", device)

  # 네트워크를 GPU로
  net.to(device)

  # 네트워크가 어느 정도 고정되면 가속화
  torch.backends.cudnn.benchmark = True

  # 반복자의 카운터 설정
  iteration = 1
  epoch_train_loss = 0.0
  epoch_val_loss = 0.0   
  logs = []

  # 에폭 Loop
  for epoch in range(num_epochs+1):

    # 시작 시간 저장
    t_epoch_start = time.time()
    t_iter_start = time.time()

    print('-------------')
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')

    # 에폭별 훈련 및 검증을 루프
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train()   # 훈련
        print('（train）')
      else:
        if((epoch+1) % 10 == 0):
          net.eval()  # 검증
          print('-------------')
          print('（val）')
        else:
          # 검증은 10회중 1회만 실시
          continue

      # 데이터 로더에서 미니 배치씩 꺼내서 루프
      for images, targets in dataloaders_dict[phase]:

        # GPU를 사용할 수 있으면 데이터 전송
        images = images.to(device)
        targets = [ann.to(device) for ann in targets]

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파 계산
        with torch.set_grad_enabled(phase == 'train'):
          # 순전파 계산
          outputs = net(images)

          # 손실 계산
          loss_l, loss_c = criterion(outputs, targets)
          loss = loss_l + loss_c

          # 훈련시에는 역전파
          if phase == 'train':
            loss.backward()  # 경사 계산

            # 경사가 너무 크면 계산이 부정확해지므로 clip에서 최대 경사를 2.0에 고정
            nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)

            optimizer.step()  # 파라미터 갱신
            
            if (iteration % 10 == 0):  # 10iter에 한번 손실 표시
              t_iter_finish = time.time()
              duration = t_iter_finish - t_iter_start
              print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(iteration, loss.item(), duration))
              t_iter_start = time.time()

              epoch_train_loss += loss.item()
              iteration += 1

            # 검증
            else:
              epoch_val_loss += loss.item()
    

    # 에폭의 페이즈 당 손실과 정답률
    t_epoch_finish = time.time()
    print('-------------')
    print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(epoch+1, epoch_train_loss, epoch_val_loss))
    print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
    t_epoch_start = time.time()

    # 로그 저장
    log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
    logs.append(log_epoch)
    df = pd.DataFrame(logs)
    df.to_csv("log_output.csv")

    epoch_train_loss = 0.0
    epoch_val_loss = 0.0

    # 네트워크 저장
    if ((epoch+1) % 10 == 0):
      torch.save(net.state_dict(), 'weights/ssd300_' + str(epoch+1) + '.pth')

In [ ]:
# 학습 및 검증 실시
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs = num_epochs)